In [ ]:
#Project Header and Imports
"""
DenseNet-121 Baseline Model for NIH Chest X-ray Classification
COSC 4368 Final Project - Group 13
Yla Herrera, Nicolas Mangilit, Kiriti Padavala, and Matt Tindall
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm

print("All libraries imported successfully!")

In [ ]:
# Set Random Seeds and Check Device
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# heck for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Cell 3: Define Paths and Load Metadata
DATA_DIR = "data" 

# The dataset has images split across multiple folders: images_001 to images_012

#Load metadata
print("Loading metadata CSV...")
csv_path = os.path.join(DATA_DIR, "Data_Entry_2017.csv")
df = pd.read_csv(csv_path)

print(f"✓ Total number of images: {len(df)}")
print(f"\nDataset columns:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
print(df.head())

#Look at image folders
print(f"\nChecking for image folders...")
image_folders = []
for i in range(1, 13):  #all folders
    folder_name = f"images_{i:03d}"
    folder_path = os.path.join(DATA_DIR, folder_name)
    if os.path.exists(folder_path):
        num_images = len([f for f in os.listdir(folder_path) if f.endswith('.png')])
        image_folders.append(folder_name)
        print(f"Found {folder_name} with {num_images} images")

print(f"\nTotal image folders found: {len(image_folders)}")


In [ ]:
#Define Disease Categories
# The 14 disease categories in NIH Chest X-ray dataset
DISEASE_CATEGORIES = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 
    'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 
    'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 
    'Pleural_Thickening', 'Hernia'
]

print(f"Number of disease categories: {len(DISEASE_CATEGORIES)}")
print(f"Disease categories: {DISEASE_CATEGORIES}")

In [ ]:
# Parse Labels and Create Binary Columns
# Parse labels (Finding Labels column contains multiple labels separated by |)
def parse_labels(label_string, disease_categories):
    """Convert label string to multi-hot encoded vector"""
    labels = np.zeros(len(disease_categories))
    if label_string != "No Finding":
        diseases = label_string.split('|')
        for disease in diseases:
            if disease in disease_categories:
                idx = disease_categories.index(disease)
                labels[idx] = 1
    return labels

# Add binary labels for each disease
for disease in DISEASE_CATEGORIES:
    df[disease] = df['Finding Labels'].apply(
        lambda x: 1 if disease in x else 0
    )

print(f"\nDisease distribution:")
print(df[DISEASE_CATEGORIES].sum().sort_values(ascending=False))

# Visualize disease distribution
plt.figure(figsize=(12, 6))
disease_counts = df[DISEASE_CATEGORIES].sum().sort_values(ascending=False)
plt.bar(range(len(disease_counts)), disease_counts.values)
plt.xticks(range(len(disease_counts)), disease_counts.index, rotation=45, ha='right')
plt.xlabel('Disease Category')
plt.ylabel('Number of Cases')
plt.title('Distribution of Diseases in Dataset')
plt.tight_layout()
plt.show()

In [ ]:
class ChestXrayDataset(Dataset):
    """Custom Dataset for NIH Chest X-rays

    image_dir may be a single path (string) or a list/tuple of directories.
    If multiple directories are provided, __getitem__ will search them in order
    for the requested image filename.
    """

    def __init__(self, dataframe, image_dir, disease_categories, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        # allow a single path or multiple roots
        if isinstance(image_dir, (list, tuple)):
            self.image_dirs = list(image_dir)
        else:
            self.image_dirs = [image_dir]

        # keep original for compatibility
        self.image_dir = image_dir
        self.disease_categories = disease_categories
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get image filename
        img_name = self.dataframe.loc[idx, 'Image Index']

        # Try each candidate root until file is found
        img_path = None
        for root in self.image_dirs:
            candidate = os.path.join(root, img_name)
            if os.path.exists(candidate):
                img_path = candidate
                break

        if img_path is None:
            # helpful message showing where we looked
            raise FileNotFoundError(f"Image '{img_name}' not found in any of: {self.image_dirs}")

        # Load image
        image = Image.open(img_path).convert('RGB')

        # Get labels (multi-hot encoded)
        labels = self.dataframe.loc[idx, self.disease_categories].values.astype(np.float32)

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels)

In [ ]:
# Cell 7: Define Data Transforms
# Define transforms for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

print("Data transforms defined:")
print(f"  - Train: Resize(224x224), RandomHorizontalFlip, ToTensor, Normalize")
print(f"  - Val: Resize(224x224), ToTensor, Normalize")

In [ ]:
# Cell 8: Train/Validation Split
# Simple train/validation split (80/20)
# For your project, you may want to use the official NIH train/val/test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.sample(n=8000, random_state=42)
val_df = val_df.sample(n=2000, random_state=42)
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"\nTrain/Val split ratio: {len(train_df)/len(df):.2%} / {len(val_df)/len(df):.2%}")

In [ ]:
# Cell 9: Create Datasets and DataLoaders
# Create datasets
DATA_DIR = "data"

# The images are split across multiple folders images_001..images_012 (each contains an inner 'images' folder).
# Build a list of directories to search for files and pass that to the dataset so it can locate each image.
IMAGE_DIRS = [os.path.join(DATA_DIR, f"images_{i:03d}", "images") for i in range(1, 13)]
# keep only existing directories (helps if some folders are missing locally)
IMAGE_DIRS = [p for p in IMAGE_DIRS if os.path.exists(p)]
print(f"Found {len(IMAGE_DIRS)} image directories; using:\n  - " + "\n  - ".join(IMAGE_DIRS))

train_dataset = ChestXrayDataset(
    train_df, 
    IMAGE_DIRS, 
    DISEASE_CATEGORIES, 
    transform=train_transform
)

val_dataset = ChestXrayDataset(
    val_df, 
    IMAGE_DIRS, 
    DISEASE_CATEGORIES, 
    transform=val_transform,
)

# Create dataloaders
# Use a smaller batch size on CPU and avoid worker processes on Windows (spawn overhead can be very slow)
BATCH_SIZE = 8  # Reduce for CPU (increase if you have more RAM/CPU)
if os.name == 'nt':
    NUM_WORKERS = 0
else:
    NUM_WORKERS = 4

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS,
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
)

print(f"DataLoaders created:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Train batches: {len(train_loader)}")
print(f"  - Val batches: {len(val_loader)}")
print(f"  - Num workers: {NUM_WORKERS}")

In [ ]:
# Diagnostic & Quick Debug Training (CPU-friendly)
# Measures DataLoader load/transfer times and runs a short 1-epoch training on a small subset
import time
from torch.utils.data import Subset

print('Device:', device)
print('Train dataset length:', len(train_dataset))
print('Train loader batches:', len(train_loader))
print('Batch size:', BATCH_SIZE, 'Num workers:', NUM_WORKERS)

# Ensure a model is available on CPU for the debug run
if 'model' not in globals():
    print("`model` not found — initializing DenseNet121 (pretrained=False) for debug.")
    try:
        model = DenseNet121(num_classes=len(DISEASE_CATEGORIES), pretrained=False)
    except NameError:
        print("`DenseNet121` class not found — building model directly from torchvision.models")
        base = models.densenet121(pretrained=False)
        num_features = base.classifier.in_features
        base.classifier = nn.Linear(num_features, len(DISEASE_CATEGORIES))
        model = base

# Move model to CPU for this debug run
model = model.to('cpu')

# Ensure a loss function exists
if 'criterion' not in globals():
    criterion = nn.BCEWithLogitsLoss()

# Use a small subset for a quick smoke test
DEBUG_SAMPLES = min(200, len(train_dataset))
subset_idx = list(range(DEBUG_SAMPLES))
debug_loader = DataLoader(Subset(train_dataset, subset_idx), batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# Measure first few batches (load + forward+loss computation)
load_times = []
step_times = []

it = iter(debug_loader)
for i in range(min(5, len(debug_loader))):
    t0 = time.time()
    imgs, lbls = next(it)
    t1 = time.time()
    load_times.append(t1 - t0)

    t2 = time.time()
    # ensure tensors on CPU (this notebook runs on CPU)
    outputs = model(imgs)
    loss = criterion(outputs, lbls)
    t3 = time.time()
    step_times.append(t3 - t2)
    print(f"Batch {i+1}: load={load_times[-1]:.3f}s, forward+loss={step_times[-1]:.3f}s")

if load_times:
    print(f"Avg load time: {sum(load_times)/len(load_times):.3f}s, avg forward+loss: {sum(step_times)/len(step_times):.3f}s")

# Quick 1-epoch training on the small subset (use CPU)
print('\nStarting short debug training (1 epoch over {} samples)'.format(DEBUG_SAMPLES))
optimizer_debug = optim.Adam(model.parameters(), lr=1e-4)

model.train()
start = time.time()
for i, (imgs, lbls) in enumerate(debug_loader, 1):
    t0 = time.time()
    optimizer_debug.zero_grad()
    outputs = model(imgs)
    loss = criterion(outputs, lbls)
    loss.backward()
    optimizer_debug.step()
    t1 = time.time()
    if i % 10 == 0 or i == len(debug_loader):
        print(f"Step {i}/{len(debug_loader)} - loss={loss.item():.4f} step_time={t1-t0:.3f}s")
end = time.time()
print(f"Debug epoch complete in {end - start:.1f}s")


In [ ]:
# Quick smoke-check: verify datasets & that an example image file exists
print('IMAGE_DIRS:', IMAGE_DIRS)
print('Train dataset length:', len(train_dataset))
print('Val dataset length:', len(val_dataset))

# Small helper to locate a filename inside the configured roots
from pathlib import Path

def find_image_path(image_name, search_dirs):
    for d in search_dirs:
        candidate = os.path.join(d, image_name)
        if os.path.exists(candidate):
            return candidate
    return None

sample_name = train_df.iloc[0]['Image Index']
print('Sample image name (from train_df):', sample_name)
found = find_image_path(sample_name, IMAGE_DIRS)
print('Found sample path:' , found)

In [ ]:
# # Cell 10: Test DataLoader (Optional - verify it works)
# # Test loading a batch
# print("Testing data loading...")
# images, labels = next(iter(train_loader))
# print(f"Batch shape: {images.shape}")
# print(f"Labels shape: {labels.shape}")
# print(f"Image value range: [{images.min():.3f}, {images.max():.3f}]")

# # Visualize a few sample images
# fig, axes = plt.subplots(2, 4, figsize=(15, 8))
# axes = axes.ravel()

# for i in range(8):
#     img = images[i].permute(1, 2, 0).numpy()
#     # Denormalize
#     img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
#     img = np.clip(img, 0, 1)
    
#     axes[i].imshow(img)
#     # Get disease labels for this image
#     disease_indices = torch.where(labels[i] == 1)[0]
#     disease_names = [DISEASE_CATEGORIES[idx] for idx in disease_indices]
#     title = ', '.join(disease_names) if disease_names else 'No Finding'
#     axes[i].set_title(title, fontsize=9)
#     axes[i].axis('off')

# plt.tight_layout()
# plt.show()
# print("Data loading test successful!")


In [ ]:
# Cell 11: Define DenseNet-121 Model
class DenseNet121(nn.Module):
    """DenseNet-121 for multi-label chest X-ray classification"""
    
    def __init__(self, num_classes=14, pretrained=True):
        super(DenseNet121, self).__init__()
        
        # Load pre-trained DenseNet-121
        self.densenet = models.densenet121(pretrained=pretrained)
        
        # Get number of input features for the classifier
        num_features = self.densenet.classifier.in_features
        
        # Replace the classifier for multi-label classification
        self.densenet.classifier = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        return self.densenet(x)

print("DenseNet121 model class defined!")

In [ ]:

# Cell 12: Initialize Model
# Initialize model
model = DenseNet121(num_classes=len(DISEASE_CATEGORIES), pretrained=True)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model initialized and moved to {device}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# Cell 13: Define Loss Function and Optimizer
# Binary Cross Entropy with Logits Loss (suitable for multi-label classification)
criterion = nn.BCEWithLogitsLoss()

# Adam optimizer
LEARNING_RATE = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2
)

print(f"Loss function: BCEWithLogitsLoss")
print(f"Optimizer: Adam (lr={LEARNING_RATE})")
print(f"Scheduler: ReduceLROnPlateau")

In [ ]:
# Cell 14: Training Function
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

print("Training function defined!")

In [ ]:
# Cell 15: Validation Function
def validate_epoch(model, dataloader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            # Store predictions and labels for metrics
            all_labels.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())
    
    epoch_loss = running_loss / len(dataloader.dataset)
    
    # Concatenate all batches
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    
    # Calculate AUC for each disease
    auc_scores = []
    for i in range(all_labels.shape[1]):
        if len(np.unique(all_labels[:, i])) > 1:  # Check if both classes present
            auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
            auc_scores.append(auc)
        else:
            auc_scores.append(np.nan)
    
    mean_auc = np.nanmean(auc_scores)
    
    return epoch_loss, mean_auc, auc_scores

print("Validation function defined!")

In [ ]:
# Cell 16: Training Loop
NUM_EPOCHS = 5  # Start with a few epochs to test

print(f"\n{'='*60}")
print(f"Starting Training - {NUM_EPOCHS} epochs")
print(f"{'='*60}\n")

train_losses = []
val_losses = []
val_aucs = []

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 40)
    
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # Validate
    val_loss, mean_auc, auc_scores = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_aucs.append(mean_auc)
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    # Print metrics
    print(f"\nTrain Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Mean AUC: {mean_auc:.4f}")
    
    print("\nPer-disease AUC scores:")
    for disease, auc in zip(DISEASE_CATEGORIES, auc_scores):
        if not np.isnan(auc):
            print(f"  {disease:25s}: {auc:.4f}")

print("\n" + "="*60)
print("Training Complete!")
print("="*60)

In [ ]:
# Cell 17: Save Model
# Save the trained model
MODEL_SAVE_PATH = "densenet121_baseline.pth"
torch.save({
    'epoch': NUM_EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'val_aucs': val_aucs,
}, MODEL_SAVE_PATH)

print(f"Model saved to {MODEL_SAVE_PATH}")

# Cell 18: Plot Training Curves
plt.figure(figsize=(15, 5))

# Loss curves
plt.subplot(1, 3, 1)
plt.plot(range(1, NUM_EPOCHS+1), train_losses, marker='o', label='Train Loss')
plt.plot(range(1, NUM_EPOCHS+1), val_losses, marker='s', label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

# AUC curve
plt.subplot(1, 3, 2)
plt.plot(range(1, NUM_EPOCHS+1), val_aucs, marker='o', color='green', label='Mean AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.title('Validation Mean AUC')
plt.legend()
plt.grid(True)

# Summary table
plt.subplot(1, 3, 3)
plt.axis('off')
summary_data = []
for i in range(NUM_EPOCHS):
    summary_data.append([i+1, f"{train_losses[i]:.4f}", f"{val_losses[i]:.4f}", f"{val_aucs[i]:.4f}"])

table = plt.table(cellText=summary_data,
                  colLabels=['Epoch', 'Train Loss', 'Val Loss', 'Mean AUC'],
                  cellLoc='center',
                  loc='center',
                  bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
plt.title('Training Summary', pad=20)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training curves saved to 'training_curves.png'")

# Cell 19: Final Results Summary
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"\nBest Validation AUC: {max(val_aucs):.4f} (Epoch {val_aucs.index(max(val_aucs))+1})")
print(f"Final Validation AUC: {val_aucs[-1]:.4f}")
print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Val Loss: {val_losses[-1]:.4f}")

print("\n" + "="*60)
print("Baseline model training complete!")
print("Next steps: Implement bias mitigation techniques")
print("="*60)